# CHAPTER 11 - The Best of Both Worlds: Hybrid Architectures - Qiskit/PyTorch Code

*Note*: You may skip the following five cells if you have alredy installed the right versions of all the libraries mentioned in *Appendix D*. This will likely NOT be the case if you are running this notebook on a cloud service such as Google Colab.

In [ ]:
pip install scikit-learn==1.2.1

In [ ]:
pip install torch==1.13

In [ ]:
pip install qiskit==0.39.2

In [ ]:
pip install qiskit_machine_learning==0.5.0

In [ ]:
pip install matplotlib==3.2.2

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

seed = 1234
np.random.seed(seed)

In [ ]:
import torch
torch.manual_seed(seed)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class TorchClassifier(nn.Module):
    
    def __init__(self):
        
        # Initialize super class.
        super(TorchClassifier, self).__init__()
        
        # Declare the layers that we will use.
        self.layer1 = nn.Linear(16, 8)
        self.layer2 = nn.Linear(8, 4)
        self.layer3 = nn.Linear(4, 2)
        self.layer4 = nn.Linear(2, 1)
    
    # Define the transformation of an input.
    def forward(self, x):
        x = F.elu(self.layer1(x))
        x = F.elu(self.layer2(x))
        x = F.elu(self.layer3(x))
        x = torch.sigmoid(self.layer4(x))
        
        return x

In [ ]:
model = TorchClassifier()
print(model)

In [ ]:
model(torch.rand(16))

In [ ]:
x, y = make_classification(n_samples = 1000, n_features = 16)

x_tr, x_test, y_tr, y_test = train_test_split(
    x, y, train_size = 0.8)
x_val, x_test, y_val, y_test = train_test_split(
    x_test, y_test, train_size = 0.5)

In [ ]:
from torch.utils.data import Dataset

class NumpyDataset(Dataset):
    def __init__(self, x, y):
        
        if (x.shape[0] != y.shape[0]):
            raise Exception("Incompatible arrays")
        
        y = y.reshape(-1,1)
        
        self.x = torch.from_numpy(x).to(torch.float)
        self.y = torch.from_numpy(y).to(torch.float)
        
    def __getitem__(self, i):
        return self.x[i], self.y[i]
    
    def __len__(self):
        return self.y.shape[0]

In [ ]:
tr_data = NumpyDataset(x_tr, y_tr)
val_data = NumpyDataset(x_val, y_val)
test_data = NumpyDataset(x_test, y_test)

In [ ]:
print(tr_data[0])
print("Length:", len(tr_data))

In [ ]:
from torch.utils.data import DataLoader
tr_loader = iter(DataLoader(
    tr_data, batch_size = 2, shuffle = True))
print(next(tr_loader))

In [ ]:
get_loss = F.binary_cross_entropy

In [ ]:
print(get_loss(torch.tensor([1.]), torch.tensor([1.])))

In [ ]:
tr_loader = DataLoader(tr_data, batch_size = 100, shuffle = True)

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr = 0.005)

In [ ]:
def run_epoch(opt, tr_loader):
    # Iterate through the batches.
    for data in iter(tr_loader):
    
        x, y = data # Get the data in the batch.
        
        opt.zero_grad() # Reset the gradients.
        
        # Compute gradients.
        loss = get_loss(model(x), y)
        loss.backward()
        
        opt.step() # Update the weights.
        
    return get_loss(model(tr_data.x), tr_data.y)

In [ ]:
tr_losses = []
val_losses = []

while (len(val_losses) < 2 or val_losses[-1] < val_losses[-2]):
    print("EPOCH", len(tr_losses) + 1, end = " ")
    tr_losses.append(float(run_epoch(opt, tr_loader)))
    # ^^ Remember that run_epoch returns the training loss.
    val_losses.append(float(
        get_loss(model(val_data.x), val_data.y)))
    print("| Train loss:", round(tr_losses[-1], 4), end = " ")
    print("| Valid loss:", round(val_losses[-1], 4))

In [ ]:
import matplotlib.pyplot as plt
def plot_losses(tr_loss, val_loss):
    epochs = np.array(range(len(tr_loss))) + 1
    plt.plot(epochs, tr_loss, label = "Training loss")
    plt.plot(epochs, val_loss, label = "Validation loss")
    plt.xlabel("Epoch")
    plt.legend()
    plt.show()

plot_losses(tr_losses, val_losses)

In [ ]:
train_acc = accuracy_score(
    (model(tr_data.x) >= 0.5).to(float), tr_data.y)
val_acc = accuracy_score(
    (model(val_data.x) >= 0.5).to(float), val_data.y)
test_acc = accuracy_score(
    (model(test_data.x) >= 0.5).to(float), test_data.y)
print("Training accuracy:", train_acc)
print("Validation accuracy:", val_acc)
print("Test accuracy:", test_acc)

In [ ]:
from qiskit import *
from qiskit.circuit.library import ZZFeatureMap, TwoLocal

In [ ]:
x, y = make_classification(n_samples = 500, n_features = 16)
x_tr, x_test, y_tr, y_test = train_test_split(x, y, train_size = 0.8)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, train_size = 0.5)

tr_data = NumpyDataset(x_tr, y_tr)
val_data = NumpyDataset(x_val, y_val)
test_data = NumpyDataset(x_test, y_test)

tr_loader = DataLoader(tr_data, batch_size = 20, shuffle = True)

In [ ]:
zzfm = ZZFeatureMap(2)
twolocal = TwoLocal(2, ['ry','rz'], 'cz', 'linear', reps = 1)

In [ ]:
from qiskit_machine_learning.neural_networks import TwoLayerQNN

In [ ]:
from qiskit_machine_learning.connectors import TorchConnector
from qiskit.providers.aer import AerSimulator

class HybridQNN(nn.Module):
    
    def __init__(self):

        # Initialize super class.
        super(HybridQNN, self).__init__()

        # Declare the layers that we will use.
        qnn = TwoLayerQNN(2, zzfm, twolocal, input_gradients = True,
            quantum_instance = AerSimulator(method="statevector"))        
        self.layer1 = nn.Linear(16, 2)
        self.qnn = TorchConnector(qnn)
        self.final_layer = nn.Linear(1,1)

    def forward(self, x):
        x = torch.sigmoid(self.layer1(x))
        x = self.qnn(x)
        x = torch.sigmoid(self.final_layer(x))
        return x

model = HybridQNN()

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr = 0.005)

In [ ]:
tr_losses = []
val_losses = []

while (len(val_losses) < 2 or val_losses[-1] < val_losses[-2]):
    print("EPOCH", len(tr_losses) + 1, end = " ")
    tr_losses.append(float(run_epoch(opt, tr_loader)))
    val_losses.append(float(get_loss(model(val_data.x), val_data.y)))
    print("| Train loss:", round(tr_losses[-1], 4), end = " ")
    print("| Valid loss:", round(val_losses[-1], 4))

In [ ]:
plot_losses(tr_losses, val_losses)

In [ ]:
tr_acc = accuracy_score(
    (model(tr_data.x) >= 0.5).to(float), tr_data.y)
val_acc = accuracy_score(
    (model(val_data.x) >= 0.5).to(float), val_data.y)
test_acc = accuracy_score(
    (model(test_data.x) >= 0.5).to(float), test_data.y)
print("Training accuracy:", tr_acc)
print("Validation accuracy:", val_acc)
print("Test accuracy:", test_acc)

*Note*: In the following cell, you need to replace "1234" with your actual IBM token. Refer to *Appendix D* in the book for instructions on how to create an account and get your token. Be very careful not to disclose your token to anyone!

In [ ]:
ibm_token = "1234"
IBMQ.save_account(ibm_token)

In [ ]:
from qiskit.providers.ibmq import *

provider = IBMQ.load_account()
dev_list = provider.backends(
    filters = lambda x: x.configuration().n_qubits >= 4,
                        simulator = False)

dev = least_busy(dev_list)

In [ ]:
class QiskitQNN(nn.Module):
    
    def __init__(self):

        super(QiskitQNN, self).__init__()

        qnn = TwoLayerQNN(2, zzfm, twolocal, input_gradients = True)
        self.qnn = TorchConnector(qnn)

    def forward(self, x):
        x = self.qnn(x)
        return x

model = QiskitQNN()

In [ ]:
x, y = make_classification(n_samples = 100, n_features = 2,
    n_clusters_per_class = 1, n_informative = 1, n_redundant = 1)
x_tr, x_test, y_tr, y_test = train_test_split(x, y, train_size = 0.8)
x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, train_size = 0.5)

tr_data = NumpyDataset(x_tr, y_tr)
val_data = NumpyDataset(x_val, y_val)
test_data = NumpyDataset(x_test, y_test)

In [ ]:
tr_loader = DataLoader(tr_data, batch_size = 20, shuffle = True)
val_loader = DataLoader(val_data)
test_loader = DataLoader(test_data)

In [ ]:
get_loss = F.mse_loss
opt = torch.optim.Adam(model.parameters(), lr = 0.005)

In [ ]:
from qiskit_machine_learning.runtime import TorchRuntimeClient

client = TorchRuntimeClient(provider = provider, backend = dev,
    model = model, optimizer = opt, loss_func = get_loss,
    epochs = 5)

In [ ]:
result = client.fit(train_loader = tr_loader, val_loader = val_loader)

In [ ]:
pred = client.predict(test_loader).prediction